In [2]:
import pandas as pd
import numpy as np
import os
# import seaborn as sns
# import plotly.express as px
# import plotly.graph_objs as go
# from plotly.tools import FigureFactory as FF
import re
import os
from datetime import datetime, timedelta 

df = pd.DataFrame()
os.getcwd()
 

'c:\\Users\\dscshap3808\\Documents\\my_scripts_new\\play4_activity'

In [ ]:
# pd.read_excel('./data/Coach11-1to11-5.xlsx', sheet_name=0)\
#     .to_csv('./data/coach_15.csv', encoding = 'utf_8_sig', index=None)
coach = pd.read_csv('./data/coach_15.csv')
coach = coach.dropna(axis=1, how = 'all')
# task_collect = coach.groupby(['姓名', '开始日期'])['任务类型'].apply(list).astype(str).reset_index();task_collect
# onoff = task_collect[task_collect['任务类型'].str.contains('(下班.+上班)')][['姓名','开始日期']].sum(axis = 1)
# coach['flag'] = coach[['姓名','开始日期']].sum(axis = 1)
coach['开始时间'] = pd.to_datetime(coach['开始时间'])
coach['结束时间'] = pd.to_datetime(coach['结束时间'])

# coach_nopunch = coach[~coach['flag'].isin(onoff)]
# coach_dopunch = coach[ coach['flag'].isin(onoff)]
coach['sprm'] = (60/coach['SPR标准数量']).replace([np.inf, -np.inf], 0)

wh = coach.groupby(['姓名','开始日期']).agg(
    {
        '开始时间': 'min',
        '结束时间': 'max',
        'sprm': 'sum',
    }
).reset_index()
wh['work_hour'] = wh['结束时间'] - wh['开始时间']
wh['work_hour_in_min']  = [i.total_seconds()/60 for i in wh['work_hour']]
wh['work_hour_in_hour'] = [i.total_seconds()/3600 for i in wh['work_hour']]
wh = wh.drop(['结束时间','开始时间', 'work_hour'], axis = 1).rename({'sprm':'SPRM_total_of_day'}, axis =1)
coach = coach.merge(wh, on = ['姓名', '开始日期'], how = 'inner')
coach['sprm_perhour'] =  coach['SPRM_total_of_day'] / coach['work_hour_in_hour']

# 标准时长计算
coach = pd.concat([coach, pd.DataFrame(list(coach['调整后持续时间'].str.split(':')))], axis =1).drop('序号',axis =1)
coach = coach[coach[[0,1,2]].astype(int).sum(axis = 1) != 0]
coach[[0,1,2]] = coach[[0,1,2]].astype(int)
coach['time_len'] = coach[0]*3600 + coach[1]*60 + coach[2]
coach = coach.drop([0,1,2], axis = 1)

In [ ]:
# coach[coach['姓名'] == '孙清元']
coach.columns

In [ ]:
# plot_data('2021-11-05').info()
# coach['开始日期'].unique()
# array(['2021-11-05', '2021-11-04', '2021-11-03', '2021-11-02',
#        '2021-11-01'], dtype=object)

In [ ]:
def plot_data(riqi):
    data = coach[coach['开始日期'] == riqi][['姓名', '工作组', '直属上级', '任职时间', '职位',\
    'sprm', 'SPRM_total_of_day', 'work_hour_in_min',
    'work_hour_in_hour', 'sprm_perhour']].drop_duplicates()
    data['工作组'] = data['工作组'].fillna('unknow')
    data['工作组'] = data['工作组'].fillna('unknow')

    return data
def plot_(riqi):
    fig2 = px.scatter(
        data_frame = plot_data(riqi), x = '姓名', y= 'sprm_perhour', hover_data=['工作组' ],
        size = 'work_hour_in_hour', color = '工作组', 
        # facet_col = '开始日期', 
        title= "报告日期" + riqi)  
    # hover_data=  ['account_name_en'] , 


    # fig2.add_hline(y = line_20, line_width=3, line_dash="dash", line_color="black", opacity = .4,\
    #     annotation_text="2020 <br> weighted mean profit_rate <br>, %s" %line_20 , 
    #     annotation_position="top right", annotation_opacity = .4, col = 1)
    # fig2.add_hline(y = line_21, line_width=3, line_dash="dash", line_color="black", opacity = .4,\
    #     annotation_text="2021 <br> weighted mean profit_rate <br>, %s" %line_21 , 
    #     annotation_position="top right", annotation_opacity = .4, col = 2)

    return fig2

plot_('2021-11-03')

In [ ]:
# SELECT  * FROM dm_dsc_smart.dwd_task
# where substr(station_name, 1,4) = 'COAC'
# and work_group_id != 0 and work_group_name != ''
# and inc_day = '20211127'
 

In [ ]:
# coach = coach.dropna(how = 'all', axis = 1)
    # time_convert()
def time_convert(col):
    coach[col] = coach[col].astype(int)
    coach[col] = [datetime.fromtimestamp(i).strftime('%Y-%m-%d %H:%M:%S') for i in coach[col]]
    return coach
for i in ['start_time', 'end_time', 'hire_time', 'update_time']:
    time_convert(i)
coach['start_time'] = pd.to_datetime(coach['start_time'])
coach['end_time']   = pd.to_datetime(coach['end_time'])
coach['sprm'] = (60/coach['work_content_refer']).replace([np.inf, -np.inf], 0)


# wh = coach.groupby(['worker_name','start_date']).agg(
#         {
#             'start_time': 'min',
#             'end_time': 'max',
#             'sprm': 'sum',
#         }
#     ).reset_index()
# wh['work_hour'] = wh['结束时间'] - wh['开始时间']
# wh['work_hour_in_min']  = [i.total_seconds()/60 for i in wh['work_hour']]
# wh['work_hour_in_hour'] = [i.total_seconds()/3600 for i in wh['work_hour']]
# wh = wh.drop(['结束时间','开始时间', 'work_hour'], axis = 1).rename({'sprm':'SPRM_total_of_day'}, axis =1)
# coach = coach.merge(wh, on = ['姓名', '开始日期'], how = 'inner')

In [ ]:
coach[coach[['duration',
    'adjustment_duration', 'adjusted_duration', ]].sum(axis = 1) != 0]

In [ ]:
# # coach= coach.drop(['dwd_task_id', 'station_id', 'work_turn_id', 'raw_data'], axis  =1)

# coach['work_content'].value_counts()
# coach['work_content_refer'].value_counts()
# coach

In [ ]:
# %%time
# df = pd.DataFrame()
# for i in np.arange(coach.shape[0]):
#     df = pd.concat([pd.DataFrame.from_dict(json.loads(coach['raw_data'][i]), orient="index" ).T, df], axis = 0)

In [ ]:
# coach[coach['start_date'] == '2021-11-23'] 
# coach

In [ ]:
def plot0_date(date, col):
    # sns.histplot(
    return coach[coach['start_date'] == date][col].value_counts()

In [ ]:
plot0_date('2021-11-28', 'work_content_refer')
# pd.set_option("display.max_rows", 15, "display.max_columns", None)
# coach.head()

In [ ]:
pd.set_option("display.max_rows", 15, "display.max_columns", None)
coach

In [ ]:
# coach = pd.read_csv('./data/coach_1202_outdata.csv', sep = '\,')
# coach = coach.dropna(how = 'all', axis = 1)
# coach.columns = [re.sub('\w+\.', '', i) for i in list(coach.columns)]
# coach = coach.dropna(how = 'all', axis = 1)

In [75]:
# coach.to_csv('./data/coach_cleaned_drop_0.csv', encoding = 'utf_8_sig')
# cica
# account_name,station_name,
# worker_name, work_turn_name,
# work_group_name,up_worker_name, hire_time, worker_post_name, worker_level_name
# ,work_num, from_unixtime(start_time), from_unixtime(end_time), duration, adjustment_duration
coach = pd.read_csv('./data/coach1129_1202.csv', sep = '\001')

In [76]:
coach

,account_name,station_id,station_code,station_name,worker_id,worker_name,work_group_id,work_group_name,up_worker_id,up_worker_name,hire_time,worker_post_id,worker_post_name,work_num,start_time,end_time,duration,adjustment_duration,adjusted_duration,sku_no,work_content,work_content_type,work_content_is_measure,work_content_refer,start_date,end_date,report_date,inc_day
0,李永生,10005659,CN-085,COACH SHA WGQ WHS,19809,李永生,10025701,拣货组,19772,孙波,1498752000,10109295,操作员,10.0,1638334905,1638334905,0,0,0,C3738 B4/TP,NaN,0,0,0,2021-12-01,2021-12-01,2021-12-01,20211201
1,李永生,10005659,CN-085,COACH SHA WGQ WHS,19809,李永生,10025701,拣货组,19772,孙波,1498752000,10109295,操作员,10.0,1638335275,1638335275,0,0,0,C3738 B4RFT,NaN,0,0,0,2021-12-01,2021-12-01,2021-12-01,20211201
2,李永生,10005659,CN-085,COACH SHA WGQ WHS,19809,李永生,10025701,拣货组,19772,孙波,1498752000,10109295,操作员,2.0,1638335905,1638336096,0,0,0,C3282 B4SHG,NaN,0,0,0,2021-12-01,2021-12-01,2021-12-01,20211201
3,李永生,10005659,CN-085,COACH SHA WGQ WHS,19809,李永生,10025701,拣货组,19772,孙波,1498752000,10109295,操作员,6.0,1638336268,1638336317,0,0,0,C5809 B4/HA,NaN,0,0,0,2021-12-01,2021-12-01,2021-12-01,20211201
4,李永生,10005659,CN-085,COACH SHA WGQ WHS,19809,李永生,10025701,拣货组,19772,孙波,1498752000,10109295,操作员,5.0,1638336340,1638336376,0,0,0,C3738 B4RFT,NaN,0,0,0,2021-12-01,2021-12-01,2021-12-01,20211201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29502,郁黄祥,10005659,CN-085,COACH SHA WGQ WHS,19805,郁黄祥,0,NaN,19775,李小丹,1588262400,10109295,操作员,12.0,1638235730,1638235730,0,0,0,C3766 B4CAH,NaN,0,0,0,2021-11-30,2021-11-30,2021-11-30,20211130
29503,郁黄祥,10005659,CN-085,COACH SHA WGQ WHS,19805,郁黄祥,0,NaN,19775,李小丹,1588262400,10109295,操作员,8.0,1638236498,1638236498,0,0,0,C4099 IMCAH,NaN,0,0,0,2021-11-30,2021-11-30,2021-11-30,20211130
29504,郁黄祥,10005659,CN-085,COACH SHA WGQ WHS,19805,郁黄祥,0,NaN,19775,李小丹,1588262400,10109295,操作员,10.0,1638236857,1638236857,0,0,0,C2586 V5C1A,NaN,0,0,0,2021-11-30,2021-11-30,2021-11-30,20211130
29505,郁黄祥,10005659,CN-085,COACH SHA WGQ WHS,19805,郁黄祥,0,NaN,19775,李小丹,1588262400,10109295,操作员,16.0,1638236857,1638236857,0,0,0,C3756 B4SHG,NaN,0,0,0,2021-11-30,2021-11-30,2021-11-30,20211130


==========================
# online script <br>
==========================

In [3]:
coach = pd.read_csv('./data/coach1129_1202.csv', sep = '\001')
coach = coach.dropna(how = 'all', axis = 1)
coach.columns = [re.sub('\w+\.', '', i) for i in list(coach.columns)]
# coach = coach.dropna(how = 'all', axis = 1)
# coach = coach.drop('raw_data', axis = 1)
# time_convert()
def time_convert(col):
    coach[col] = coach[col].astype(int)
    coach[col] = [datetime.fromtimestamp(i).strftime('%Y-%m-%d %H:%M:%S') for i in coach[col]]
    return coach
for i in ['start_time', 'end_time', 'hire_time']:
    time_convert(i)
coach['start_time'] = pd.to_datetime(coach['start_time'])
coach['end_time']   = pd.to_datetime(coach['end_time'])
# coach['duration'] =  coach['end_time'] - coach['start_time']
coach = coach[~coach['work_content'].isna()]
coach = coach[coach['work_content'] != '无效时间']
coach = coach[coach['worker_post_name'] != '操作经理']
# sprm calculation
coach['sprm'] = (60/coach['work_content_refer']).replace([np.inf, -np.inf], 0)

wh = coach.groupby([
    'worker_name','inc_day', 'station_name', 'work_group_name',
    ]).agg(
    {
        'start_time': 'min',
        'end_time': 'max',
        'duration':'sum',
        'adjusted_duration': 'sum',
        'sprm': 'sum',
    }
).reset_index()

wh['work_hour'] = wh['end_time'] - wh['start_time']
# wh['work_hour_in_min']  = [i.total_seconds()/60 for i in wh['work_hour']]
wh['tt_work_hour'] = [i.total_seconds()/3600 for i in wh['work_hour']]
wh['tt_adj_duration_in_hour']  = wh['adjusted_duration']/3600
wh = wh.drop(['end_time','start_time', 'work_hour'], axis = 1)
wh = wh.rename({'sprm':'tt_sprm', 'duration' : 'tt_duration', 'adjusted_duration': 'tt_adj_duration' }, axis =1 )
wh = wh[wh['tt_sprm'] != 0]
wh['sprm_perhour'] =  wh['tt_sprm'] / wh['tt_work_hour']

In [86]:
smt_wt = pd.read_csv('./data/dws_dsc_smart_work_type_dtl_df.csv', sep = '\001')
smt_wt.columns = [re.sub('\w+\.', '', i) for i in list(smt_wt.columns)]

smt_wt.merge(wh, on = ['station_name', 'worker_name', 'inc_day', 'work_group_name'], how = 'left')

In [7]:
df['sprm_hour_day'] = df.groupby('worker_name')['tt_sprm'].transform('mean') \
        / df.groupby('worker_name')['tt_work_hour'].transform('mean')

In [13]:
df[df['worker_name'] == '吴飞宇']

,worker_name,inc_day,station_name,work_group_name,tt_duration,tt_adj_duration,tt_sprm,tt_work_hour,tt_adj_duration_in_hour,sprm_perhour,sprm_hour_day
0,吴飞宇,20211129,COACH SHA WGQ WHS,库存组,11632,11632,348.0,9.0,3.231111,38.666667,37.555556
1,吴飞宇,20211130,COACH SHA WGQ WHS,库存组,8083,8083,328.0,9.0,2.245278,36.444444,37.555556


In [14]:
df

,worker_name,inc_day,station_name,work_group_name,tt_duration,tt_adj_duration,tt_sprm,tt_work_hour,tt_adj_duration_in_hour,sprm_perhour,sprm_hour_day
0,吴飞宇,20211129,COACH SHA WGQ WHS,库存组,11632,11632,348.000000,9.000000,3.231111,38.666667,37.555556
1,吴飞宇,20211130,COACH SHA WGQ WHS,库存组,8083,8083,328.000000,9.000000,2.245278,36.444444,37.555556
2,宋卫华,20211129,COACH SHA WGQ WHS,库存组,944,944,65.833333,11.940833,0.262222,5.513295,2.407526
3,宋卫华,20211130,COACH SHA WGQ WHS,库存组,23,23,1.333333,9.000000,0.006389,0.148148,2.407526
4,宋卫华,20211201,COACH SHA WGQ WHS,库存组,124,124,12.166667,12.011389,0.034444,1.012928,2.407526
5,李小丹,20211130,COACH SHA WGQ WHS,库存组,14613,14613,2820.000000,13.000000,4.059167,216.923077,249.772727
6,李小丹,20211201,COACH SHA WGQ WHS,库存组,6959,6959,2675.000000,9.000000,1.933056,297.222222,249.772727
7,李文博,20211130,COACH SHA WGQ WHS,发货组,85,85,7.750000,9.000000,0.023611,0.861111,0.861111
8,豆富山,20211130,COACH SHA WGQ WHS,VAS组,1218,1218,75.333333,9.000000,0.338333,8.370370,8.589582
9,豆富山,20211201,COACH SHA WGQ WHS,VAS组,3673,3673,102.666667,11.722778,1.020278,8.757879,8.589582


In [72]:
def data_prepare(coach):

    """
    time unix convert, 
    转换后标准时长换算成秒,
    sprm 计算.
    工作在勤时间
    """
    coach = pd.read_csv('./data/coach1129_1202.csv', sep = '\001')
    coach = coach.dropna(how = 'all', axis = 1)
    coach.columns = [re.sub('\w+\.', '', i) for i in list(coach.columns)]
    # coach = coach.dropna(how = 'all', axis = 1)
    # coach = coach.drop('raw_data', axis = 1)
    # time_convert()
    def time_convert(col):
        coach[col] = coach[col].astype(int)
        coach[col] = [datetime.fromtimestamp(i).strftime('%Y-%m-%d %H:%M:%S') for i in coach[col]]
        return coach
    for i in ['start_time', 'end_time', 'hire_time']:
        time_convert(i)
    coach['start_time'] = pd.to_datetime(coach['start_time'])
    coach['end_time']   = pd.to_datetime(coach['end_time'])
    # coach['duration'] =  coach['end_time'] - coach['start_time']
    coach = coach[~coach['work_content'].isna()]
    coach = coach[coach['work_content'] != '无效时间']
    coach = coach[coach['worker_post_name'] != '操作经理']
    # sprm calculation
    coach['sprm'] = (60/coach['work_content_refer']).replace([np.inf, -np.inf], 0)
   
    wh = coach.groupby(['worker_name','inc_day', 'station_name', 'work_group_name']).agg(
        {
            'start_time': 'min',
            'end_time': 'max',
            'sprm': 'sum',
        }
    ).reset_index()
    
    wh['work_hour'] = wh['end_time'] - wh['start_time']
    wh['work_hour_in_min']  = [i.total_seconds()/60 for i in wh['work_hour']]
    wh['work_hour_in_hour'] = [i.total_seconds()/3600 for i in wh['work_hour']]
    wh = wh.drop(['end_time','start_time', 'work_hour'], axis = 1).rename({'sprm':'SPRM_total_of_day'}, axis =1)
    
    # coach = coach.merge(wh, on = ['worker_name', 'inc_day', 'station_name'], how = 'inner')
    # coach = coach[coach['duration'] != '0']
    coach = coach[coach['sprm'] != 0]
    coach['sprm_perhour'] =  coach['SPRM_total_of_day'] / coach['work_hour_in_hour']
    coach = coach[coach['work_content'] != '无效时间']
    coach = coach[coach['worker_post_name'] != '操作经理']
    """
    计算转换后时间长度, 换算成 
    秒
    """
    # coach = pd.concat([coach, pd.DataFrame(list(coach['adjusted_duration'].str.split(':')))], axis =1) 
    # coach = coach[coach[[0,1,2]].astype(int).sum(axis = 1) != 0]
    # coach[[0,1,2]] = coach[[0,1,2]].astype(int)
    # coach['time_len'] = coach[0]*3600 + coach[1]*60 + coach[2]
    # coach = coach.drop([0,1,2], axis = 1) 
    
    return coach
coach = data_prepare(coach)


In [95]:
# coach[coach['work_hour_in_hour'] == coach['work_hour_in_hour'].max()] 
# coach.head(10) 

In [ ]:
np.argwhere(np.array(tst == 193))

In [ ]:
coach = coach.sort_values(['worker_name', 'start_time', 'end_time'])
shift = coach[['worker_name', 'start_time', 'end_time']]
shift = shift.groupby(['worker_name']).shift(1)
shift.columns = ['start_shift1', 'end_shift1']

In [ ]:
test = pd.concat([coach, shift], axis = 1)
test['gap'] = test['start_time'] - test['end_shift1'] 

In [ ]:
# test[test['gap'] > '0 days 00:00:00']

In [ ]:
test = coach[(coach['inc_day'].astype(str).str.contains('20211130')) & (coach['worker_name'] == '张现花')]

In [ ]:
# def time_convert(coach,col):
#     coach[col] = coach[col].astype(int)
#     coach[col] = [datetime.fromtimestamp(i).strftime('%Y-%m-%d %H:%M:%S') for i in coach[col]]
#     return coach
# for i in ['start_time', 'end_time', 'hire_time']:
#     time_convert(test,i)

In [ ]:
pd.to_datetime(test['end_time']).min()

In [ ]:
pd.to_datetime(test['end_time']).max() - pd.to_datetime(test['end_time']).min()

In [ ]:
test[test['start_time'] == '2021-11-30 08:57:23'] 

In [ ]:
wh = test.groupby(['worker_name','inc_day']).agg(
        {
            'start_time': 'min',
            'end_time': 'max',
            'sprm': 'sum',
        }
    ).reset_index()
    
wh['work_hour'] = wh['end_time'] - wh['start_time']

In [ ]:
wh 

In [ ]:
[i.total_seconds() for i in wh['work_hour']][0]/3600

In [ ]:
# coach[(coach['start_time'].astype(str).str.contains('2021-11-30')) & (coach['worker_name'] == '郁黄祥')]

In [ ]:
coach[(coach['worker_name'] == '郁黄祥') & (coach['start_date'] == '2021-11-30')]['start_time'].min()

In [ ]:
coach[(coach['worker_name'] == '郁黄祥') & (coach['start_date'] == '2021-12-01')]

In [ ]:
# =======

In [ ]:
coach_out = coach[['worker_name',  'work_group_name', 'up_worker_name', 'hire_time',\
         'worker_post_name', 'SPRM_total_of_day', 'work_hour_in_min',
         'work_hour_in_hour', 'sprm_perhour', 'station_name', 'inc_day']].drop_duplicates()
coach_out['inc_day'] = coach_out['inc_day'].astype(str)
        #  'sprm', 'SPRM_total_of_day', 'work_hour_in_min',
        #  'work_hour_in_hour', 'sprm_perhour',

In [ ]:
coach_out[coach_out['work_hour_in_hour'] == 13]

In [ ]:
test  = coach.query("worker_name == '李小丹'") [['worker_name',  'work_group_name', 'up_worker_name', 'hire_time',\
         'start_time', 'end_time','inc_day']].drop_duplicates() 

In [ ]:
te2 = test.groupby(['worker_name','inc_day']).agg(
        {
            'start_time': 'min',
            'end_time': 'max'
        }
    ).reset_index() 

In [ ]:
te2['end_time'] - te2['start_time']

In [ ]:
pd_loaddata.pd_tocsv(coach_out, './data/coach_1202_outdata.csv' )

In [ ]:
from fyenn_class import pd_loaddata
pd_loaddata.show_line(15,None)
# coach[(coach['account_name'] == '孙清元') & (coach['start_date'] == '2021-11-30')].sort_values('start_time')

In [ ]:
px.bar(data_frame =  x = 'worker_name', y = 'sprm_perhour', color = 'work_group_name')

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
def plot_data(coach, riqi, site):
    data = coach[coach['station_name'].str.contains(site)]
    data = coach[coach['inc_day'].astype(str) == riqi].drop_duplicates()
    data['work_group_name'] = data['work_group_name'].fillna('unknown')
    data = data.sort_values('sprm_perhour', ascending = False)
    return data.drop_duplicates()
def plot_(coach, riqi, site):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    data = plot_data(coach, riqi, site)
    for i in data['work_group_name'].unique():
        data_0 = data[data['work_group_name'] == i]
        fig.add_trace(go.Scatter(
            x = data_0['worker_name'], y = data_0['work_hour_in_hour'], name = 'working_hour',
            showlegend=False),
            secondary_y = True)
        # fig.update_layout(showlegend=False)
        fig.add_trace(go.Bar(
            x = data_0['worker_name'], y = data_0['sprm_perhour'], name = i, 
            # marker = dict(colorbar = data['work_group_name'], autocolorscale=True)
            ),
            secondary_y = False) 
    # fig.update_layout(
    #         title=go.layout.Title(text="工时 & sprm/h; 时间:" + riqi + '\t'+ coach['station_name'][0])
    #         )
    fig.update_yaxes(title_text="<b>sprm/h</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>working_hour</b>", secondary_y=True)
    return fig

plot_(coach_out, '20211130', 'COA')

In [ ]:
plot_(coach_out, '20211130','COA').write_html('./data/c20211130.html')

=================
# already pulished

In [ ]:
coach_bdp_published_1205 = pd.read_csv('./data/coach_bdp_published.csv', sep = '\001')

In [ ]:
coach_bdp_published_1205.columns = [re.sub('\w+\.','', str(i)) for i in list(coach_bdp_published_1205.columns)]

In [ ]:
# coach_bdp_published_1205

In [ ]:
# pd.concat([plot_data(coach_bdp_published_1205, '20211205', 'COA'), plot_data(coach_bdp_published_1205, '20211204', 'COA')], axis = 0).to_csv('./data/coach_1204_1205.csv', index = None)

In [ ]:
plot_(coach_bdp_published_1205, '20211204', 'COA').write_html('./data/c20211204.html')

In [ ]:
# plot_data(coach_bdp_published_1205, '20211204', 'COA')

In [ ]:
# =====

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
def plot_data(coach, riqi, site):
    data = coach[coach['station_name'].str.contains(site)]
    data = coach[coach['inc_day'].astype(str) == riqi].drop_duplicates()
    data['work_group_name'] = data['work_group_name'].fillna('unknown')
    data = data.sort_values('sprm_perhour', ascending = False)
    return data.drop_duplicates()
def plot_(coach, riqi, site):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    data = plot_data(coach, riqi, site)
    for i in data['work_group_name'].unique():
        data_0 = data[data['work_group_name'] == i]
        fig.add_trace(go.Scatter(
            x = data_0['worker_name'], y = data_0['work_hour_in_hour'], name = 'working_hour',
            showlegend=False),
            secondary_y = True)
        # fig.update_layout(showlegend=False)
        fig.add_trace(go.Bar(
            x = data_0['worker_name'], y = data_0['sprm_perhour'], name = i, 
            # marker = dict(colorbar = data['work_group_name'], autocolorscale=True)
            ),
            secondary_y = False) 
    # fig.update_layout(
    #         title=go.layout.Title(text="工时 & sprm/h; 时间:" + riqi + '\t'+ coach['station_name'][0])
    #         )
    fig.update_yaxes(title_text="<b>sprm/h</b>", secondary_y=False)
    fig.update_yaxes(title_text="<b>working_hour</b>", secondary_y=True)
    return fig

plot_(coach_out, '20211130', 'COA')